In [ ]:
# Imports
import requests
from requests.auth import HTTPBasicAuth
from dateutil.parser import parse as parse_date
import pandas as pd
from config import jira_pat as pat

In [ ]:
# Set Up Code With Your Parameters

# owner is used to construct the urls for the jira api to access issues
owner_url = '<the jira instance url>'
project = '<jira project name>'
authorizer_email = "<email associated with your PAT>"

# the date range you're interested in
start_date = '2023-08-01'
end_date = '2023-10-31'
start_week = parse_date(start_date).isocalendar().week
end_week = parse_date(end_date).isocalendar().week

In [ ]:
# The api call setup

url = f"https://{owner_url}/rest/api/3/search"
jql = f"project = \"{project}\" and status = Done AND resolved >= \"{start_date}\" and resolved <= \"{end_date}\""

# This is how you add custom fields to your request. You'll have to change the NNNNN to the id in your project
investment_type = 'customfield_NNNNN'
story_points = 'customfield_NNNNN'

fields = [
  "key",
  "status",
  story_points,
  investment_type,
  "issuetype",
  "labels"
]

print(jql)

In [ ]:
# Make requests (this may take some time)

params = {
  'jql': jql,
  'validateQuery': 'strict',
  'fields': fields,
  'maxResults': 50,
}

auth = HTTPBasicAuth(authorizer_email, pat)

headers = {
  "Accept": "application/json"
}

issues = []
page = 0
while(True):
  print(f'fetching page {page}')
  params["startAt"] = params["maxResults"]*page
  r = requests.get(url, headers=headers, params=params, auth=auth)
  issues.extend(r.json()["issues"])
  if len(issues) < r.json()["total"]:
    page += 1
    continue
  break

print(f"\nTotal number of issues: {len(issues)}")

In [ ]:
def recursive_extract_nested_data(data_dict, complex_key):
  keys = complex_key.rsplit('.')
  if len(keys) == 1:
    return data_dict[keys[0]]
  return recursive_extract_nested_data(data_dict[keys[0]], '.'.join(keys[1:]))

transformed_data = {}

for issue in issues:
  issue_data = {
    "issueType": recursive_extract_nested_data(issue, 'fields.issuetype.name'),
    "status": recursive_extract_nested_data(issue, 'fields.status.name'),
    "investmentType": recursive_extract_nested_data(issue, f'fields.{investment_type}.value'),
    "storyPoints": recursive_extract_nested_data(issue, f'fields.{story_points}'),
    "labels": recursive_extract_nested_data(issue, f'fields.labels')
  }
  transformed_data[issue["key"]] = issue_data

print(transformed_data)

In [ ]:
# Create dataframe

df = pd.DataFrame(transformed_data).T
df.head()

In [ ]:
# Create dataframes for plots

issues_by_investment_type_df = df.groupby(['investmentType'])['storyPoints'].sum()
issues_by_investment_type_df.head()

In [ ]:
# Plot investment type
title = f"Investment type breakdown between {start_date} and {end_date}"

issues_by_investment_type_df.plot.pie(title=title, ylabel='', autopct='%1.1f%%', shadow=True, startangle=0)

In [ ]:
issues_by_issue_type_df = df.groupby(['issueType'])['storyPoints'].sum().drop('Epic')
issues_by_issue_type_df.head()

In [ ]:
# Plot issue type
title = f"Issue type breakdown between {start_date} and {end_date}"

issues_by_issue_type_df.plot.pie(title=title, ylabel='', autopct='%1.1f%%', shadow=True, startangle=0)